In [ ]:
# # This file is part of Theano Geometry
#
# Copyright (C) 2017, Stefan Sommer (sommer@di.ku.dk)
# https://bitbucket.org/stefansommer/theanogemetry
#
# Theano Geometry is free software: you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.
#
# Theano Geometry is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.
#
# You should have received a copy of the GNU General Public License
# along with Theano Geometry. If not, see <http://www.gnu.org/licenses/>.
#

# Manifold Statistics - Examples on $\mathbb{S}^2$

In [ ]:
from src.manifolds.S2 import *
M = S2()
print(M)

from src.plotting import *

In [ ]:
# Riemannian structure
from src.Riemannian import metric
metric.initialize(M)

# geodesics
from src.Riemannian import geodesic
geodesic.initialize(M)

# Logarithm map
from src.Riemannian import Log
Log.initialize(M)

## Sample Data

In [ ]:
# coordinate form
from src.stochastics import Brownian_coords
Brownian_coords.initialize(M)

N_samples = 20
x = np.zeros((M.dim.eval()))

samples = np.zeros((N_samples,M.dim.eval()))
for i in range(N_samples):
    (ts,xs) = M.Brownian_coordsf(x,dWsf(M.dim.eval()))
    samples[i] = xs[-1]

# plot
newfig()
M.plot()
for i in range(N_samples):
    M.plotx(samples[i])
plt.show()

## Frechet mean

In [ ]:
from src.statistics.Frechet_mean import *

res = Frechet_mean(M,lambda *args: M.Logf(*args), samples)
Fm = res[0]
print("loss = ", res[1])
print("mean = ", Fm)
iterations = res[2]

newfig()
M.plot(rotate = np.array([50,-45]))
M.plotx(Fm)
M.plotx(iterations)
plt.show()

## Frechet mean on FM

In [ ]:
n_samp = 5
y0 = np.random.normal(0., 0.2, (n_samp,2))

x0 = np.array([0.4,0.]).astype(theano.config.floatX)
ui0 = np.array([1,0,0,1])
q0 = np.hstack([x0,ui0.flatten()]).astype(theano.config.floatX)
print(q0)

v0 = np.array([-1.,0.]).astype(theano.config.floatX)
v0 = GramSchmidt(v0,x0)
p0 = gMflatf(x0,v0)
#xia0 = np.zeros(4)
#p0 = np.hstack([xi0,xia0])
print(p0)

from src.FM import *

qsv = Expfm(q0,p0)

In [ ]:
from src.FM import *

detg = lambda x,u: T.nlinalg.Det()(T.tensordot(u.T,T.tensordot(g(x),u,axes=(1,0)),axes=(1,0)))

def loss(m,dat): 
    
    u = m[0:(d+rank*d)]
    v = m[(d+rank*d):].reshape((n_samp,d))
    
    (cout, updates) = theano.scan(fn = lambda v,q: Expfm(q,gMflat(q,v))[0:d],
                                  sequences = [v],
                                  non_sequences = [u],
                                  n_steps=n_samp)
    
    #T.sum(h_v**2)
    return cout #1./n_samp*(T.sum(v**2) + lambda0*T.sum((cout - dat)**2)) - 1./2*T.log()

m = T.vector() # frame and horizontal vectors
dat = T.matrix()
lossf = theano.function([m,dat], loss(m,dat))

In [ ]:
y0 = np.random.normal(0., 0.2, (n_samp,2))

q0 = np.array([0.,0.]).astype(theano.config.floatX)
v0 = y0 - q0


In [ ]:
print(data.shape)
print(q0.shape)
print(p0.shape)
def Log(x0, y, Fr):
    def fopts(x):
#         print(Ttype(np.vstack([concatx(x0,Fr.flatten()),x])),)
        y = lossf(np.vstack([concatx(x0,Fr.flatten()),x]),1./n_steps.eval(),n_steps.eval())
        return y

    res = minimize(fopts, np.zeros([d.eval()+rank.eval()*d.eval()]), method='CG', jac=False, options={'disp': False, 'maxiter': 50})
    return res.x
Logf(q0,data[i,:],p0)

In [ ]:
from src.Hamiltonian import *

data = y0
q0 = np.array([0.3,0.]).astype(theano.config.floatX)
p0 = np.array([1,0,0,1])
u0 = np.hstack((q0,p0))

#Logyis = np.zeros((Nsamples,d+d*rank))
openPool()

def Log(x0, y, Fr):
    def fopts(x):
#         print(Ttype(np.vstack([concatx(x0,Fr.flatten()),x])),)
        y = lossf(np.vstack([concatx(x0,Fr.flatten()),x]),1./n_steps.eval(),n_steps.eval())
        return y

    res = minimize(fopts, np.zeros([d.eval()+rank.eval()*d.eval()]), method='CG', jac=False, options={'disp': False, 'maxiter': 50})
    return res.x

print("shooting for initial Logs")
def lLog(i):
    return Log(q0, data[i,:], p0)
print(lLog(1))
#sol = pool.imap(lLog, zip(*(np.arange(n_samp),)) )
#print(sol)
#Logyis = np.array(list(sol)) 

closePool()

#MPPLogf(1,u0)

In [ ]:
# unconstrained penalized version
from scipy.optimize import minimize,fmin_bfgs,fmin_cg,fmin_l_bfgs_b,approx_fprime, check_grad
import

def DiffSource(data, tol=1e-4, maxIter=20, x0=None, Xa0=None, Logyis=None):

    def MPPLogf(idx,m):
        Nsamples = (m.shape[0]-(d+d*rank))/(d+d*rank)
        q0 = 1e-2*m[0:d+d*rank]
        Logsamples = m[d+d*rank:].reshape([Nsamples,d+d*rank])

        f = lambda xx: lossf(Ttype(np.vstack([concatx(x0,Xa0.flatten()),xx])),1./n_steps0,n_steps0)
        y = f(Logsamples[idx,:])

        return (y,)

    def dMPPLogf(idx,m):
        Nsamples = (m.shape[0]-(d+d*rank))/(d+d*rank)
        x0 = 1e-2*m[0:d]
        Xa0 = 1e-2*m[d:d+d*rank].reshape([d,rank])
        Logsamples = m[d+d*rank:].reshape([Nsamples,d+d*rank])
        
        qtarget0.set_value(Ttype(data[idx,:]))
        f = lambda xx: lossf(Ttype(xx).reshape([2,d+rank*d]),1./n_steps0,n_steps0)
        gy = approx_fprime(np.vstack([concatx(x0,Xa0.flatten()),Logsamples[idx,:].flatten()]).flatten(),f,1e-5)
    
        return (gy,)
    
    
    def f(m):
        # energy
        Nsamples = (m.shape[0]-(d+d*rank))/(d+d*rank)

        EHs = np.empty(Nsamples)
        x0 = 1e-2*m[0:d]
        Xa0 = 1e-2*m[d:d+d*rank].reshape([d,rank])
        Logsamples = m[d+d*rank:].reshape([Nsamples,d+d*rank])
        
        for idx in range(Nsamples):
            xi = Logsamples[idx,0:d]
            Xia = Logsamples[idx,d:d+d*rank].reshape([d,rank])            
            EHs[i] = Hsplitf(x0,Xa0,xi,Xia)

        res = (1./Nsamples)*np.sum(EHs)
        detXa02 = detg2f(x0,Xa0)
        Xa02inner = np.einsum('ba,bi,ij->aj',Xa0,gf(x0),Xa0)
#         print("f x0: %s, Xa02 diag: %s, Xa02 off: %s, det: %g, res %g, logstuff %g" % (x0,np.diag(Xa02inner),Xa02inner[0,1],detXa02,res,(1./2.)*np.log(detXa02)))
#         print("f Xa02 diag: %s, Xa02 off: %s, det: %g, res %g, logstuff %g" % (np.diag(Xa02inner),Xa02inner[0,1],detXa02,res,(1./2.)*np.log(detXa02)))
        return 1e0*(res + .5*np.log(detXa02))
    
    def constr(m):
        # parallel compute distances        
        Nsamples = (m.shape[0]-(d+d*rank))/(d+d*rank)

        sol = pool.imap(MPPLogf, zip(*(xrange(Nsamples), itertools.cycle((m,)))) )
        res = list(sol)
        Logs = getRes(res,0)

#         res = 1e-3-(1./Nsamples)*np.sum(Logs)
        res = 1e-5-Logs
        return res

    def dconstr(m):
        # parallel compute distances        
        Nsamples = (m.shape[0]-(d+d*rank))/(d+d*rank)

        sol = pool.imap(dMPPLogf, zip(*(xrange(Nsamples), itertools.cycle((m,)))) )
        res = list(sol)
        dLogsres = getRes(res,0)
        dLogs = np.zeros([Nsamples,m.shape[0]])
        for i in range(Nsamples):
            dLogs[i,0:d+rank*d] = 1e-2*dLogsres[i,0:d+d*rank]
            dLogs[i,d+rank*d+i*(d+rank*d):d+rank*d+(i+1)*(d+rank*d)] = dLogsres[i,d+d*rank:2*(d+rank*d)]
        

#         res = 1e-3-(1./Nsamples)*np.sum(Logs)
#         res = 1e-4-np.sum(Logs)
        dres = -dLogs
        return dres
            
        
    # number of samples
    Nsamples = data.shape[0]
            
    if x0 == None:
        # initial point
        x0 = np.mean(data,0)
    if Xa0 == None:
        # initial frame        
        Gx0M = gf(x0)
        Xa0 = np.eye(d)
        Xa0 = np.linalg.solve(sp.linalg.sqrtm(Gx0M),Xa0) # make orthonormal

    print("initial point/frame, x0: %s, Xa0: %s" % (x0,Xa0))
    
    openPool()
    
    if Logyis == None:
        #Logyis = np.zeros((Nsamples,d+d*rank))
        print("shooting for initial Logs")
        def lLog(i):
            return Log(x0, data[i,:], Fr)
        sol = pool.imap(lLog, zip(*(xrange(Nsamples),)) )
        Logyis = np.array(list(sol))    

    initval = np.hstack( (1e2*x0,1e2*Xa0.flatten(),Logyis.flatten(),) )
    
#     print("checking constr gradient")
#     for i in range(Nsamples):
#         err = check_grad(lambda x: constr(x)[i],lambda x: dconstr(x)[i],initval)
#         print("sampe %i contr grad erro %g" % (i,err))
#         if err > 1:
#             print(dconstr(initval)[i])
#             print(approx_fprime(initval,lambda x: constr(x)[i],1e-5))
    
    print("running constrained optimization")
    res = minimize(f, initval, method='SLSQP',\
                   tol=tol,\
                   constraints={'type': 'ineq', 'fun': constr, 'jac': dconstr},\
                   options={'disp': True, 'maxiter': maxIter},\
                   )
    closePool()
    
    if not res.success:
        print("mean/covar optimization failed:\n%s" % res)
    mu = 1e-2*res.x[0:d]
    SigmaSQRT = 1e-2*res.x[d:d+d*rank]
    Logyis = res.x[d+d*rank:].reshape([Nsamples,d+d*rank])

    return (mu,SigmaSQRT,Logyis)

In [ ]:
DiffSource(y0, tol=1e-4, maxIter=20, x0=None, Xa0=None, Logyis=None)

## Brownian motions

In [ ]:
from src.Brownian_Stochastic_Development import *

In [ ]:
# Observation
x0 = np.array([0.,0.]).astype(theano.config.floatX)

# Frame
v0 = np.array([[0.4,0],[0.,0.4]]).astype(theano.config.floatX)
ui0 = v0 #GramSchmidt(v0,x0) #sp.linalg.orth(v0) # Gram-Schmidt

q0 = np.hstack([x0,ui0.flatten()]).astype(theano.config.floatX)

n_steps.set_value(1000)
dWt0 = np.random.normal(0, np.sqrt(dt.eval()), (n_steps.get_value(),2))

In [ ]:
Wt0 = np.cumsum(dWt0, axis=0)

%matplotlib inline
plt.plot(np.hstack((0,Wt0[:,0])),np.hstack((0,Wt0[:,1])),'b-',
        linewidth=1)
plt.plot(0,0,'ro',markersize=10.)
plt.plot(Wt0[-1,0],Wt0[-1,1],'go',markersize=10.)
#plt.savefig("Brown1.pdf")

In [ ]:
# Brownian motion:
Bt = SD_brownian(q0,2,dWt0)
#%matplotlib inline
plt.rcParams['figure.figsize'] = 10, 10
newfig()
plotM(rotate = np.array([50,-45]))
plotFMx(Bt)
plt.show()
#plt.savefig("Brown.pdf")

## Normal Distributions

In [ ]:
# Observation
x0 = np.array([0.,0.]).astype(theano.config.floatX)

# Frame
v0 = np.array([[0.1,0.2],[0.2,0.2]]).astype(theano.config.floatX)
ui0 = v0#0.3*GramSchmidt(v0,x0) #sp.linalg.orth(v0) # Gram-Schmidt
print(ui0)
q0 = np.hstack([x0,ui0.flatten()]).astype(theano.config.floatX)

In [ ]:
from src.normal_dist import *
a = normal_dist_sample(5000,q0)[:,0:2]

In [ ]:
#%matplotlib notebook
#plt.rcParams['figure.figsize'] = 10, 10

aa = np.zeros((a.shape[0],3))
for i in range(a.shape[0]):
    aa[i,:] = Ff(a[i,:])
    
newfig()
plotM(rotate = np.array([55,0]))
plot_density_estimate(aa, alpha = 0.9)
#plt.show()
#plt.savefig("Anisonorm.pdf")